In [2]:
import xlrd
import os
from pandas import Series,DataFrame
import numpy as np
import pandas as pd

###
#### 1、通过header=None的方式读取excel

In [3]:
file_name='data/convert_2.xlsx'
read_data=pd.read_excel(file_name,sheetname=0,header=None)

In [5]:
read_data.head()

,0,1,2,3,4,5,6
0,年 月 日,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
1,NaN,NaN,主班,副班,警报班,主班,NaN
2,2017-12-01 00:00:00,刘秋兴,韦锋余,徐瑞,王娟娟,唐茂宁,刘琼
3,2017-12-02 00:00:00,刘秋兴,邢闯,韦锋余,徐瑞,唐茂宁,刘琼
4,2017-12-03 00:00:00,刘秋兴,侯放,邢闯,韦锋余,唐茂宁,刘琼


In [6]:
test_data=pd.DataFrame(np.arange(6).reshape(2,3))
test_data

,0,1,2
0,0,1,2
1,3,4,5


* 读取行标题

In [7]:
read_data.columns.tolist()

[0, 1, 2, 3, 4, 5, 6]

#### 1.1 去掉第一列，取剩下的前两行

In [8]:
# 会出错 KeyError: 1
# read_data[1]
read_data[:2]

,0,1,2,3,4,5,6
0,年 月 日,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
1,NaN,NaN,主班,副班,警报班,主班,NaN


#### 1.2获取第一列起始至结束；第0行起始至第一行的数据

In [9]:
columns_names=read_data.iloc[0:2,1:]
columns_names

,1,2,3,4,5,6
0,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
1,NaN,主班,副班,警报班,主班,NaN


In [10]:
columns_names.shape[1]

6

In [11]:
np.arange(columns_names.shape[1])

array([0, 1, 2, 3, 4, 5])

#### 1.3 修改columns

In [14]:
columns_names.columns=np.arange(columns_names.shape[1])

In [15]:
columns_names

,0,1,2,3,4,5
0,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
1,NaN,主班,副班,警报班,主班,NaN


* Nan不能通过==np.nan的方式判断是否为空
* 只能通过is np.nan的方式

In [16]:
print(columns_names[1][1])
type(columns_names[1][1])

主班


str

In [17]:
print(np.nan)
type(np.nan)

nan


float

In [18]:
columns_names[1][1]==np.nan

False

In [19]:
columns_names[1][1] is np.nan

False

* pandas中若要获取dataframe中是nan的位置可以使用isnull，来获取矩阵

In [20]:
columns_names==np.NaN

,0,1,2,3,4,5
0,False,False,False,False,False,False
1,False,False,False,False,False,False


In [21]:
columns_names=='NaN'

,0,1,2,3,4,5
0,False,False,False,False,False,False
1,False,False,False,False,False,False


In [22]:
is_null=columns_names.isnull()
is_null

,0,1,2,3,4,5
0,False,False,False,False,False,False
1,True,False,False,False,False,True


### 2 使用where找到isnull的位置

In [23]:
columns_names

,0,1,2,3,4,5
0,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
1,NaN,主班,副班,警报班,主班,NaN


In [24]:
columns_names.shape

(2, 6)

#### 2.1 获取bool数组

In [25]:
index_null=np.where(columns_names.isnull())
index_null

(array([1, 1], dtype=int64), array([0, 5], dtype=int64))

In [26]:
# 注意index_null是一个tuple
print(type(index_null))
index_null

<class 'tuple'>


(array([1, 1], dtype=int64), array([0, 5], dtype=int64))

#### 2.2 转置

In [27]:
# 需要转换为ndarray
# 并且需要转置
index_array=np.array(index_null).T
index_array

array([[1, 0],
       [1, 5]], dtype=int64)

##### （1）尝试获取nan所在的位置的值

In [28]:
index_array[0]

array([1, 0], dtype=int64)

In [30]:
# 行号
index_row=index_array[0][0]
# 列号
index_columns=index_array[0][1]
print(index_row)
print(index_columns)

1
0


In [31]:
print(index_array[0])
# 这样是获取列为1和0的两列
columns_names[index_array[0]]

[1 0]


,1,0
0,海浪组,风暴潮组
1,主班,NaN


* dataframe索引 df[val]：选取df的单个列或一组列

In [32]:
columns_names[index_columns][index_row]

nan

##### （1）获取nan所在的位置方式（二）：

##### （2）取null的右侧的值

In [154]:
columns_names[index_columns+1][index_row]

'主班'

In [97]:
len(index_array)

2

In [82]:
np.where(columns_names.isnull())[0]

array([1, 1], dtype=int64)

In [83]:
np.where(columns_names.isnull())[1]

array([0, 5], dtype=int64)

In [69]:
columns_names[index_null[0]][index_null[1]+1]

,1
0,风暴潮组
1,NaN


In [72]:
columns_names[2][1]

'主班'

* 先找右侧的值，若不存在再用左侧的值填充

In [163]:
columns_names

,0,1,2,3,4,5
0,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
1,NaN,主班,副班,警报班,主班,NaN


* 对于dataframe使用索引的方式第一个值是列（列的起始值是从0开始）；第二个值是行号（行号是从0开始）

In [129]:
print(columns_names[4][1])
print(columns_names[6][1])
print(columns_names[1][1])

警报班
nan
nan


In [164]:
columns_names[1][0]

'海浪组'

In [166]:
columns_names[1][1]

'主班'

In [167]:
'''

'''

def dumpingNull(df):
    # 找到isnull的位置
    index_null=np.where(df.isnull())
    index_array=np.array(index_null).T
#     print(columns_names.shape[1])
    # 遍历二维数组
    count=len(index_array)
    for temp in index_array:
        # 行号
        index_row=temp[0]
        # 列号
        index_columns=temp[1]
        print(temp[1]+1>columns_names.shape[1])
        '''
            注意与df的索引方式正好相反
            df是先列，再行
        '''         
        # 行号
        print("为null的位置[%s,%s]"%(temp[0],temp[1]))
        # 列号
#         print(temp[1])
        print("为null的值是%s"%df[temp[1]][temp[0]])        
        # 若相加之后的值没有（也就是当前null的位置是最右侧的位置）
#         if index_columns>=columns_names.shape[1]-1 and df[temp[1]-1][temp[0]] is not np.nan:  
        if index_columns>=columns_names.shape[1]-1:        
            value_replace=df[index_columns-1][index_row]
            print("右侧没有值")
#             print("要替换的位置[%s,%s]"%(temp[1],temp[0]))
            print("要替换的值为%s"%value_replace)
        else:
            print("右侧还有值")
            print("替换的值的位置[%s %s]"%(index_columns+1,temp[0]))
            value_replace=df[index_columns+1][temp[0]]
#             print("要替换的位置[%s,%s]"%(temp[1]+1,temp[0]))
            print(value_replace)
        print("-------")
#         value_right=df[temp[0]][temp[1]+1]
#         print(value_right)
#         print(temp[0])
#         print(temp[1])
dumpingNull(columns_names)

False
为null的位置[1,0]
为null的值是nan
右侧还有值
替换的值的位置[1 1]
主班
-------
False
为null的位置[1,5]
为null的值是nan
右侧没有值
要替换的值为主班
-------


In [36]:
def dumpingNan(df):
    def replaceNan(df,index_colunms,index_rows):
        '''
        index_colunms:列
        inde_rows:行
        '''
        # 此处可能存在的问题是由于是or 所以是否需要两个条件都需要进行判断，若第一个判断为true的话第二个条件进行判断时会出错
        if index_colunms>=df.shape[1]-1 or df[index_colunms+1][index_row] is np.nan:
            # 若右侧补齐超出边界，或右侧值为null，则进行左侧补齐
            if index_colunms<=0 or df[index_colunms-1][index_rows] is np.nan:
                # 位置左移
                index_colunms-=1
                # 递归
                df=replaceNan(df,index_colunms,index_rows)
            else:
                df[index_colunms][index_rows]=df[index_colunms-1][index_rows]
        else:
            df[index_colunms][index_rows]=df[index_colunms+1][index_rows]
        return df
    # 找到isnull的位置
    index_null=np.where(df.isnull())
    index_array=np.array(index_null).T
#     print(columns_names.shape[1])
    # 遍历二维数组
    count=len(index_array)
    for temp in index_array:
        # 行号
        index_row=temp[0]
        # 列号
        index_columns=temp[1]
        
        '''
            注意与df的索引方式正好相反
            df是先列，再行
        '''         
        df=replaceNan(df,index_row,index_columns)

result=dumpingNan(columns_names)
result

KeyError: 5

#### 转换日期

In [20]:
import time

In [21]:
time.gmtime(43072.0)

time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=11, tm_min=57, tm_sec=52, tm_wday=3, tm_yday=1, tm_isdst=0)

In [22]:
targetdate=86640+43072.0
targetdate

129712.0

In [23]:
time.gmtime(targetdate)

time.struct_time(tm_year=1970, tm_mon=1, tm_mday=2, tm_hour=12, tm_min=1, tm_sec=52, tm_wday=4, tm_yday=2, tm_isdst=0)